In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q transformers

In [4]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW

device = 'cuda' if torch.cuda.is_available() else 'cpu'

import textwrap

In [5]:
# Загружаем токенайзер модели
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [6]:
# Загружаем файл с текстом, на котором будем дообучать модель
import re
path_to_txt = '/content/sobranie-stihotvoreniy-i-poem.txt'
with open(path_to_txt, encoding='utf8') as f:
    text = f.read()

text = re.sub('\n{2,}', '\n', text)
print(text[:1000])

      
      За отчие сакли у скальных снегов, Заветных лугов многоцветные шали Мы встали на битву со сворой врагов, Что жить помешали, напав по-шакальи.
      Спешили паршивые псы устрашить, Сон мирной отчизны жестоко нарушив, Любовь задушить, очаги потушить, Отваги и чести лишить наши души.
      Обычай храня, оседлал я коня, Обняв, попрощался с тобой у аула. Под ливень огня провожая меня, Украдкою мама слезинку смахнула.
      Утешь мать мою: смерть в далеком краю Зловеще крыла надо мной не расплещет. Я клятву даю: нет, не дрогнет в бою Душа, что при встрече с тобою трепещет.
      Мне жизнь дорога. Но родные луга, Седые снега, смех детишек в ауле И нашу любовь сберегу от врага И сердцем горячим закрою от пули.
      
      *
      
      Свой путь начав, не знаю чувства страха. Сквозь чад коптилки вижу по ночам, Как на плечах могучего Хунзаха Звенит арча, горит родной очаг,
      И камни скал над бурными реками, И лед вершин, и беркутов полет, Испытанный веками и клинками Жизнелюби

In [7]:
# токенизируем текст
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

Token indices sequence length is longer than the specified maximum sequence length for this model (561039 > 2048). Running this sequence through the model will result in indexing errors


561039


array([ 6202,   225,   372,  6202,   225,   999,   360,  2624, 26507,
         326])

In [8]:
# разбиваем на train и test

l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

561039 448824 112206


In [9]:
from transformers import GPT2LMHeadModel

# Эту модель просто подгружаем и не будем дообучать
model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)


# Эту модель подгрузим и далее обучим
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [10]:
batch_size = 8
max_len = 256
epochs = 5

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения
# эффективности градиентного спуска
total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(
                                            optimizer,
                                            num_warmup_steps = 3,
                                            num_training_steps = total_steps)


# зададим точность, хотя ориентироваться будем на качество генерации
def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

219 54


In [11]:
def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad() # optimizer.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None,
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        # print(logits)
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)


    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():
            loss, logits, _ = model(batch_ids,
                                token_type_ids=None,
                                labels=batch_ids
                                 ).values()

        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/5 : training


  0%|          | 0/219 [00:00<?, ?it/s]

epoch 1/5 : validation


  0%|          | 0/54 [00:00<?, ?it/s]

epoch 2/5 : training


...

  0%|          | 0/54 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/219 [00:00<?, ?it/s]

epoch 5/5 : validation


  0%|          | 0/54 [00:00<?, ?it/s]

In [12]:
logits.shape

torch.Size([8, 256, 50264])

In [13]:
# prompt – строка, которую примет на вход и продолжит
prompt = """Я пошла"""

# токенизируем строку
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)

# out будет содержать результаты генерации в виде списка
out = model_init.generate(
    # входная строка
    input_ids=prompt,
    # максимальная длина генерируемой последовательности
    max_length=70,
    # num_beams
    num_beams=5,
    # применяем сэмплирование
    do_sample=True,
    # применяем температуру
    temperature=1.,
    # топ слов по вероятности
    top_k=50,
    # топ слов по суммарной вероятности
    top_p=0.6,
    # сколько (постараться) не повторять n_gram подряд
    no_repeat_ngram_size=3,
    # сколько вернуть генераций
    num_return_sequences=3,
    ).cpu().numpy()

# out содержит результаты

In [14]:
# декодируем и печатаем
for out_ in out:
    print(tokenizer.decode(out_))

Я пошла на работу.

— А ты? — спросила я. — Что ты делаешь?

Она пожала плечами:

		Я иду на работу, а ты…

Я посмотрела на нее и увидела, что она не знает, что ответить. Я не знала, что сказать. Я просто смотрела на нее
Я пошла на работу.

— А ты? — спросила я. — Что ты делаешь?

Она пожала плечами:

		Я иду на работу, а ты…

Я посмотрела на нее и увидела, что она не знает, что ответить. Я не знала, что сказать. Я сказала: «Я
Я пошла на работу.

— А ты? — спросила я. — Что ты делаешь?

Она пожала плечами:

		Я иду на работу, а ты…

Я посмотрела на нее и увидела, что она не знает, что ответить. Я не знала, что сказать, и не могла.



In [18]:
# дообученная модель
with torch.inference_mode():
    prompt = """Я пошёл в лес"""
    prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
    out = model.generate(
        input_ids=prompt,
        max_length=100,
        num_beams=10,
        do_sample=True,
        temperature=2.,
        top_k=500,
        top_p=0.75,
        no_repeat_ngram_size=3,
        num_return_sequences=3,
        ).cpu().numpy()
    for out_ in out:
        print(textwrap.fill(tokenizer.decode(out_), 40), end='\n------------------\n')

Я пошёл в лес.       Я люблю тебя, моя
страна родная, И когда ты встаешь, и
встаешь ли – На рассвете, когда не
знаешь, что делать, Не знаю, кто я –
человек или песчинка в горах.     Я иду,
мой снег, по долине далекой И вижу, как
солнце в долине горит. Я слышу, как
звенит ручей. И я иду, и не слышу, Как
птицы поют, и что они говорят, Меняясь
между собой
------------------
Я пошёл в лес.       Я люблю тебя, моя
страна родная, И когда ты встаешь, и
встаешь ли – На рассвете, когда не
знаешь, что делать, Не знаю, кто я –
человек или песчинка в горах.     Я иду,
мой снег, по долине далекой И вижу, как
солнце в долине горит. Я слышу, как
звенит ручей. И я иду, и не слышу, Как
птицы поют, и что они говорят, Меняясь
не в
------------------
Я пошёл в лес.       Я люблю тебя, моя
страна родная, И когда ты встаешь, и
встаешь ли – На рассвете, когда не
знаешь, что делать, Не знаю, кто я –
человек или песчинка в горах.     Я иду,
мой снег, по долине далекой И вижу, как
солнце в долине горит. Я слышу

In [19]:
torch.save(model.state_dict(), '/content/drive/MyDrive/gamzatov_weights.pt')